In [1]:
import sys
import os

import tensorflow as tf
print('TensorFlow:{}'.format(tf.__version__))
import numpy as np
print('NumPy:{}'.format(np.__version__))

import matplotlib.pyplot as plt

from tensorflow.examples.tutorials.mnist import input_data

current_path = os.getcwd()
base_dir = os.path.dirname(current_path)

if not base_dir in sys.path:
    sys.path.append(base_dir)

print(sys.path)

%reload_ext autoreload
%autoreload 2
import datasetslib

mnist_home = os.path.join(datasetslib.datasets_root, 'mnist')
mnist = input_data.read_data_sets(mnist_home, one_hot=True)

X_train = mnist.train.images
X_test = mnist.test.images
y_train = mnist.train.labels
y_test = mnist.test.labels

num_outputs = 10
num_inputs = 784

tf.reset_default_graph()

# 多层感知机
def mlp(x, num_inputs, num_outputs, num_layers, num_neurons):
    w = []
    b = []
    for i in range(num_layers):
        # 权重
        w.append(tf.Variable(tf.random_normal(
            [num_inputs if i==0 else num_neurons[i-1], num_neurons[i]]), 
                             name='w_{0:04d}'.format(i)
                            ))

        # 偏差
        b.append(tf.Variable(tf.random_normal([num_neurons[i]]), name='b_{0:04d}'.format(i)
                            ))
        
    w.append(tf.Variable(tf.random_normal(
        [num_neurons[num_layers-1] if num_layers > 0 else num_inputs, num_outputs]),
                        name='w_out'))
    b.append(tf.Variable(tf.random_normal([num_outputs]), name='b_out'))
    
    # x是输入层
    layer = x
    # 添加隐藏层
    for i in range(num_layers):
        layer = tf.nn.relu(tf.matmul(layer, w[i]) + b[i])
    # 添加输出层
    layer = tf.matmul(layer, w[num_layers]) + b[num_layers]
    
    return layer

def mnist_batch_func(batch_size=100):
    x_batch, y_batch = mnist.train.next_batch(batch_size)
    return [x_batch, y_batch]


def tensorflow_classification(n_epochs, n_batches, batch_size, batch_func, model, optimizer, loss, accuracy_function, X_test, Y_test):
    with tf.Session() as tfs:
        tfs.run(tf.global_variables_initializer())
        for epoch in range(n_epochs):
            epoch_loss = 0.0
            for batch in range(n_batches):
                X_batch, Y_batch = batch_func(batch_size)
                feed_dict = {x: X_batch, y: Y_batch}
                _, batch_loss = tfs.run([optimizer, loss], feed_dict)
                epoch_loss += batch_loss
            average_loss = epoch_loss / n_batches
            print('epoch {0:04d} loss = {1:.6f}'.format(epoch, average_loss))
        feed_dict = {x: X_test, y: Y_test}
        accuracy_score = tfs.run(accuracy_function, feed_dict=feed_dict)
        print('accuracy = {0:.8f}'.format(accuracy_score))

        
tf.reset_default_graph()
    
x = tf.placeholder(dtype=tf.float32, shape=[None, num_inputs], name='x')
y = tf.placeholder(dtype=tf.float32, shape=[None, num_outputs], name='y')

num_layers = 0
num_neurons = []
learning_rate = 0.01
n_epochs = 50
batch_size = 100
n_batches = int(mnist.train.num_examples/batch_size)

model = mlp(x=x,
            num_inputs=num_inputs,
            num_outputs=num_outputs,
            num_layers=num_layers,
            num_neurons=num_neurons)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)

prediction_check = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
accuracy_function = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

tensorflow_classification(n_epochs=n_epochs,
                         n_batches=n_batches,
                         batch_size=batch_size,
                         batch_func=mnist_batch_func,
                         model=model,
                         optimizer=optimizer,
                         loss=loss,
                         accuracy_function=accuracy_function,
                         X_test=mnist.test.images,
                         Y_test=mnist.test.labels)

TensorFlow:1.13.1
NumPy:1.16.3
['c:\\python36\\python36.zip', 'c:\\python36\\DLLs', 'c:\\python36\\lib', 'c:\\python36', '', 'c:\\python36\\lib\\site-packages', 'c:\\python36\\lib\\site-packages\\pip-9.0.1-py3.6.egg', 'c:\\python36\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\WJ\\.ipython', 'D:\\Work\\tensorflow_test']
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./datasets\mnist\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./datasets\mnist\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./datasets\mnist\t10k-images-idx3-ubyte.gz
Extracting ./datasets\mnist\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such

In [2]:
num_layers = 1
num_neurons = []
for i in range(num_layers):
    num_neurons.append(8)
    
learning_rate = 0.01
n_epochs = 50
batch_size = 100
n_batches = int(mnist.train.num_examples/batch_size)

model = mlp(x=x,
            num_inputs=num_inputs,
            num_outputs=num_outputs,
            num_layers=num_layers,
            num_neurons=num_neurons)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)

prediction_check = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
accuracy_function = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

tensorflow_classification(n_epochs=n_epochs,
                         n_batches=n_batches,
                         batch_size=batch_size,
                         batch_func=mnist_batch_func,
                         model=model,
                         optimizer=optimizer,
                         loss=loss,
                         accuracy_function=accuracy_function,
                         X_test=mnist.test.images,
                         Y_test=mnist.test.labels)

epoch 0000 loss = 3.411459
epoch 0001 loss = 2.210572
epoch 0002 loss = 2.022875
epoch 0003 loss = 1.890177
epoch 0004 loss = 1.779281
epoch 0005 loss = 1.688049
epoch 0006 loss = 1.613588
epoch 0007 loss = 1.551283
epoch 0008 loss = 1.498727
epoch 0009 loss = 1.453876
epoch 0010 loss = 1.414699
epoch 0011 loss = 1.380326
epoch 0012 loss = 1.349421
epoch 0013 loss = 1.320288
epoch 0014 loss = 1.294798
epoch 0015 loss = 1.270293
epoch 0016 loss = 1.247314
epoch 0017 loss = 1.225995
epoch 0018 loss = 1.204541
epoch 0019 loss = 1.184105
epoch 0020 loss = 1.163835
epoch 0021 loss = 1.143601
epoch 0022 loss = 1.123723
epoch 0023 loss = 1.104597
epoch 0024 loss = 1.085514
epoch 0025 loss = 1.066914
epoch 0026 loss = 1.048135
epoch 0027 loss = 1.031393
epoch 0028 loss = 1.013850
epoch 0029 loss = 0.996959
epoch 0030 loss = 0.980240
epoch 0031 loss = 0.964548
epoch 0032 loss = 0.948476
epoch 0033 loss = 0.933188
epoch 0034 loss = 0.918711
epoch 0035 loss = 0.904572
epoch 0036 loss = 0.890275
e

In [3]:
num_layers = 2
num_neurons = []
for i in range(num_layers):
    num_neurons.append(256)
    
learning_rate = 0.01
n_epochs = 50
batch_size = 100
n_batches = int(mnist.train.num_examples/batch_size)

model = mlp(x=x,
            num_inputs=num_inputs,
            num_outputs=num_outputs,
            num_layers=num_layers,
            num_neurons=num_neurons)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)

prediction_check = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
accuracy_function = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

tensorflow_classification(n_epochs=n_epochs,
                         n_batches=n_batches,
                         batch_size=batch_size,
                         batch_func=mnist_batch_func,
                         model=model,
                         optimizer=optimizer,
                         loss=loss,
                         accuracy_function=accuracy_function,
                         X_test=mnist.test.images,
                         Y_test=mnist.test.labels)

epoch 0000 loss = 58.404377
epoch 0001 loss = 12.860040
epoch 0002 loss = 7.908385
epoch 0003 loss = 5.658149
epoch 0004 loss = 4.232787
epoch 0005 loss = 3.346118
epoch 0006 loss = 2.709890
epoch 0007 loss = 2.239875
epoch 0008 loss = 1.866765
epoch 0009 loss = 1.617455
epoch 0010 loss = 1.375186
epoch 0011 loss = 1.191594
epoch 0012 loss = 1.034016
epoch 0013 loss = 0.901348
epoch 0014 loss = 0.793413
epoch 0015 loss = 0.714670
epoch 0016 loss = 0.634422
epoch 0017 loss = 0.562480
epoch 0018 loss = 0.491850
epoch 0019 loss = 0.436744
epoch 0020 loss = 0.384074
epoch 0021 loss = 0.347512
epoch 0022 loss = 0.309014
epoch 0023 loss = 0.276722
epoch 0024 loss = 0.248843
epoch 0025 loss = 0.226315
epoch 0026 loss = 0.205495
epoch 0027 loss = 0.183316
epoch 0028 loss = 0.161380
epoch 0029 loss = 0.148160
epoch 0030 loss = 0.134875
epoch 0031 loss = 0.121845
epoch 0032 loss = 0.105294
epoch 0033 loss = 0.097690
epoch 0034 loss = 0.087702
epoch 0035 loss = 0.081179
epoch 0036 loss = 0.071706

In [5]:
num_layers = 3
num_neurons = []
for i in range(num_layers):
    num_neurons.append(8)
    
learning_rate = 0.01
n_epochs = 50
batch_size = 100
n_batches = int(mnist.train.num_examples/batch_size)

model = mlp(x=x,
            num_inputs=num_inputs,
            num_outputs=num_outputs,
            num_layers=num_layers,
            num_neurons=num_neurons)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)

prediction_check = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
accuracy_function = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

tensorflow_classification(n_epochs=n_epochs,
                         n_batches=n_batches,
                         batch_size=batch_size,
                         batch_func=mnist_batch_func,
                         model=model,
                         optimizer=optimizer,
                         loss=loss,
                         accuracy_function=accuracy_function,
                         X_test=mnist.test.images,
                         Y_test=mnist.test.labels)

epoch 0000 loss = 2.746121
epoch 0001 loss = 2.223680
epoch 0002 loss = 2.172618
epoch 0003 loss = 2.139252
epoch 0004 loss = 2.105271
epoch 0005 loss = 2.071610
epoch 0006 loss = 2.052745
epoch 0007 loss = 2.027435
epoch 0008 loss = 2.005989
epoch 0009 loss = 1.989630
epoch 0010 loss = 1.968230
epoch 0011 loss = 1.950378
epoch 0012 loss = 1.933832
epoch 0013 loss = 1.918492
epoch 0014 loss = 1.897512
epoch 0015 loss = 1.890226
epoch 0016 loss = 1.872777
epoch 0017 loss = 1.860911
epoch 0018 loss = 1.846063
epoch 0019 loss = 1.837533
epoch 0020 loss = 1.822538
epoch 0021 loss = 1.815060
epoch 0022 loss = 1.806092
epoch 0023 loss = 1.794899
epoch 0024 loss = 1.785806
epoch 0025 loss = 1.778228
epoch 0026 loss = 1.768430
epoch 0027 loss = 1.761350
epoch 0028 loss = 1.751501
epoch 0029 loss = 1.746616
epoch 0030 loss = 1.741719
epoch 0031 loss = 1.731736
epoch 0032 loss = 1.726825
epoch 0033 loss = 1.718465
epoch 0034 loss = 1.720384
epoch 0035 loss = 1.703633
epoch 0036 loss = 1.701392
e